# 🔻 [Return to workflow](#leftoff)

⚓ ANCHOR FOR RETURN TO WORKFLOW LINK <a name="leftoff"></a>

# 🏡 **AirBNB Dataset Review** 🏨

# ❌ Update target audience and guiding questions

---

**Who?**
>* 🏢 **AirBNB Corporate** interested in maximizing customer satisfaction to increase repeat guests and encourage new guests to stay with AirBNB hosts
>
>
>* 🏡**AirBNB hosts** interested in maximizing the ratings

**Why?**
>* 💰 **Revenue Management:** 
>
>
>
>* 🤝 **Sales:**
>
>
>
>* 🛌 **Rooms Ops:**

>
>
>

**What?**
>* 🧾 Dataset comprised of... 
>  * different features
>  * reservation records
>  * Source cited in Readme

❌ **How?**
>* Which models/methods?
>* Data prep and feature engineering

---

# 🎯  **Goal:**

Determining whether or not a host location would receive a score greater than or equal to 4/5 (defined by `'review_scores_rating'`).

# 📌 **To-Do**

---

- [ ] [TD1](#td1)
- [ ] [TD2](#td2)
- [ ] [TD3](#td3)
- [ ] [todo4](#td4)
- [ ] [todo5](#td5)
- [ ] [todo6](#td6)
- [ ] [todo7](#td7)

---

# 📂 **Imports and Settings**

In [ ]:
## Data Handling
import pandas as pd
import numpy as np
from scipy import stats


## Visualizations
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact_manual
import missingno

## Modeling - SKLearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn import set_config
set_config(display='diagram')

from imblearn.over_sampling import SMOTE,SMOTENC

# from sklearn.naive_bayes import MultinomialNB # for naive bayes model

## Settings
%matplotlib inline
plt.style.use('seaborn-talk')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('max_rows', 100)

In [ ]:
## Personal functions
import clf_functions.functions as cf
%load_ext autoreload
%autoreload 1
%aimport clf_functions.functions

## ✅ Show Visualizations Setting

In [ ]:
## Controlling whether or not to show visualizations
show_visualizations = False

## ❓ FSDS

In [ ]:
# import fsds as fs

In [ ]:
# fs.ihelp_menu([fs.ihelp_menu, sort_report])

# 📖 **Read Data**

In [ ]:
## Reading data and saving to a DataFrame

source = 'data/listings.csv.gz'

data = pd.read_csv(source)

In [ ]:
## Inspecting imported dataset
data.head(5)

In [ ]:
## Checking number of rows and columns
data.shape

---

> The initial read of the dataset shows there are 74 features and 8,033 entries. A quick glance at the `.head()` gives a sample of the entries, showing that some of the features are not relevant to my analysis.
>
> I need to get a better idea of the statistics for the dataset, especially any missing values and the datatypes for each column. I need to pre-process this data before I can perform any modeling.

---

# 👨‍💻 **Interactive Investigation**

---

> To increase accessibility to the data, **I include a widget to allow the user to sort through the data interactively.** I use [**Jupyter Widgets**](https://ipywidgets.readthedocs.io/en/latest/index.html) to create this interactive report.
>
>**To use:** select which column by which you would like to sort from the dropdown menu, then click the "Run Interact" button.
>
>***Note about 'Drop_Cols' and Cols:*** these keyword arguments are used to allow the user to drop specific columns.
>
> **Only click the "Drop_Cols" option when specifying "Cols"!** Otherwise it will cause an error.
>
>The 'Cols' dropdown menu does not affect the resulting report; the data is filtered from the report prior to displaying the results. 
>
>I chose to include this option for flexibility and adaptability, but it does have the unintended consequence of creating another drop-down menu. Please ignore this menu, as it does not provide any additional functionality. For future work, I will disable the menu to prevent confusion.

---

In [ ]:
## Running report on unfiltered dataset

interact_manual(cf.sort_report, Sort_by=list(cf.report_df(data).columns),
                Source=source);

In [ ]:
data.head(3)

---

> After reviewing my data, I see there are several features that contain irrelevant entries (URLs, source data, meta data) or values that are too complicated for simple processing (such as host and listing descriptions).
>
> I will drop these columns for the second report to review the remaining data for further processing.

---

In [ ]:
## Specifying columns to drop

drop = ['id', 'host_id', 'name', 'description', 'neighborhood_overview', 'host_name',
        'host_about', 'host_location', 'neighbourhood', 'property_type',
        'listing_url', 'scrape_id', 'last_scraped', 'picture_url','host_url',
        'host_thumbnail_url','host_picture_url','calendar_last_scraped']

In [ ]:
## Creating updated interactive report

interact_manual(cf.sort_report, Drop_Cols = True, Cols = drop,
                Sort_by=list(cf.report_df(data).columns), Source=source);

---

> **Interpretation:**
>
> The report shows that the dataset has a big problem with missing values:
>
> * **Empty:**
>   * `neighbourhood_group_cleansed`
>   * `bathrooms`
>   * `calendar_updated`
>
>
> * **Nearly empty:**
>  * `license`
>
>
> * **Missing 26-39% of data:**
>  * `host_about`
>  * `neighborhood_overview`
>  * `neighbourhood`
>  * `host_response_time`
>  * `host_response_rate`
>  * `review_scores_value`
>  * `review_scores_checkin`
>  * `review_scores_location`
>  * `review_scores_accuracy`
>  * `review_scores_communication`
>  * `review_scores_cleanliness`
>  * `host_acceptance_rate`
>  * `reviews_per_month`
>  * `first_review`
>  * `review_scores_rating`
>  * `last_review`
>
>---
>
> I will need to address these missing values before processing with the modeling. A few options include:
>
> * **Filling with the string "missing"** to indicate the value was missing.
>    * *I would be able to treat "missing" as a distinct category and use it for modeling as well.*
>
>
> * **Dropping the rows with missing values.**
>    * *This may negatively impact the accuracy of my results by overfitting to the training data.*
>
>
> * I could **use the `SimpleImputer` tool from SKLearn to fill the missing values** with the mean, median, or mode values for each.
>    * *I could couple this with a `GridSearchCV` to identify the method that has the strongest positive impact on my classification metrics.*

---

---

> To get a better idea of the missing values, I create a visual of the values via the 'Missingno' package. This visualization package includes several options for visualizing the missing data.

---

In [ ]:
## Visually inspecting missing values
if show_visualizations == True:
    missingno.bar(data, labels=True);

---

> Based on this visualization, I see that **there is a consistent trend in missing values for review scores:** if a row is missing one review score, it seems to be missing all of them.
>
> Additionally, **there are many missing values for the response time, response rate, and acceptance rate.** I want to use these columns in my classification, so I will need to replace those missing values.
>
> After reviewing these details, **I feel more comfortable with the option of dropping those rows with missing review values.** I will drop the values as part of my overall classification process.

---

# 🪓 **Train/Test Split**

---

> Before I run any further pre-processing, I split my data into training and test sets to allow me to test my model's performance.
>
> **In order to split my classification target feature properly, I will convert the original values to binary values.** Since my goal is to determine whether or not a given host property will have a high score (4+), I assign all values greater-than or equal-to 4 to '1' and anything less than 4 as '0.'
>
> **This conversion also allows me to use the "stratify" parameter in my train/test split,** which will preserve the class balance when I split my data. This will be key for proper evaluation of my models.

---

In [ ]:
## Using np.select to reassign target values based on conditional evaluations

cond = [data['review_scores_rating'] >= 4,
        data['review_scores_rating'] < 4
       ]

choice = [1,0]

data['review_scores_rating'] = np.select(cond, choice, 0)

In [ ]:
## Reviewing results to confirm only 0/1 values
data['review_scores_rating'].value_counts(dropna=False)

In [ ]:
## Creating features/target for dataset
target = 'review_scores_rating'

X = data.drop(columns = target).copy()
y = data[target].copy()

In [ ]:
## Confirming same number of rows
X.shape[0] == y.shape[0]

In [ ]:
## Splitting to prevent data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                    stratify = y)

# 🧼 **Data Cleaning and EDA**

## 🔎 Fixing Missing Values

---

> This dataset is missing a significant number of values for different columns. **In order to perform any modeling, I will need to address these missing values first.**
>
> Depending on the feature and the number of missing values per row, I will take different approaches to keep as much data as possible and in its original state.

---

In [ ]:
# Dropping features with high percentages (25%+) of missing values

drop_na_cols = []
for col in X_train.columns:
    if ((X_train[col].isna().sum()) / len(X_train[col])) > .25 and col != 'review_scores_rating':
        drop_na_cols.append(col)

drop_na_cols

In [ ]:
## Appending previous list of columns to drop (metadata, etc.)

for col in drop:
    if col not in drop_na_cols:
        drop_na_cols.append(col)

drop_na_cols

In [ ]:
## Creating new dataframe that does not include the features to drop
X_train = X_train.drop(columns= drop_na_cols).copy()
X_train

In [ ]:
## Confirming dropped columns with high missing values
cf.report_df(X_train)

In [ ]:
## Filling missing values for 'beds' with values for 'bedrooms'

for idx in list(X_train['beds'][X_train['beds'].isna()].index):
    if X_train['bedrooms'][idx] > 0:
        X_train['beds'][idx] = X_train['bedrooms'][idx]

In [ ]:
## Filling missing values for 'bedrooms' with values for 'beds'

for idx in list(X_train['bedrooms'][X_train['bedrooms'].isna()].index):
    if X_train['beds'][idx] > 0:
        X_train['bedrooms'][idx] = X_train['beds'][idx]

In [ ]:
## Confirming reduction in missing values for 'beds' and 'bedrooms'

rpt_clean  = cf.report_df(X_train)
rpt_clean[rpt_clean['null_sum'] >0]

In [ ]:
## Checking remaining missing values

X_train.isna().sum()

In [ ]:
## Removing rows with 6+ null values

X_train = X_train[X_train.isna().sum(axis=1) < 6]
X_train.head(5)

In [ ]:
X_train.isna().sum()

In [ ]:
cf.report_df(X_train)

In [ ]:
## Resetting the index after dropping rows

X_train.reset_index(drop=True, inplace=True)

In [ ]:
print(len(X_train) == len(X_train.index),"\n")
print(len(X_train),len(X_train.index))

---

> At this point, **I cleaned up most of the null values via dropping columns with 25%+ missing values and dropping rows with 6+ missing values.**
>
>Additionally, **I filled missing values for 'beds'/'bedrooms' by checking the missing values for each column against the values in the other for each row.** If a row had a value in one of the columns but not the other, I filled the missing value with the value from the other column.
>
> At this point, I addressed most of the missing values in my dataset by dropping columns and filling missing values. There are still a few columns with missing values, but I will use a SimpleImputer combined with a GridSearchCV to determine the best method by which to fill those values.
>
> Now I will review the remaining data and determine if there are any other issues with my data.

---

In [ ]:
len(X_train) == len(X_train.index)

# **COMMENT:** What else to clean?? 

* DONE: T/F columns to 1/0


* DONE: 'host_since' to DT


* DONE: 'price' -$, to float


* DONE: 'neighbourhood_cleansed' split on ", " and convert to binary columns, then drop host_neighbourhood


* DONE: 'bathrooms_text' split on space, keep 1st part, convert to int


* 'host_verifications' - single string, needs extensive work in order to MLB

## Converting True/False Columns to Binary Values

In [ ]:
## Creating list of true/false features to convert to 1/0, respectively

t_f_xf = ['host_is_superhost','host_has_profile_pic','host_identity_verified',
          'has_availability','instant_bookable']
t_f_xf

In [ ]:
## Converting datatype to "string" to replace values

X_train[t_f_xf] = X_train[t_f_xf].astype('str')
X_train[t_f_xf].dtypes

In [ ]:
X_train[t_f_xf]

In [ ]:
## Converting t/f to 1/0, respectively

X_train[t_f_xf] = X_train[t_f_xf].replace({ 't' : 1, 'f' : 0})

In [ ]:
X_train[t_f_xf]

In [ ]:
X_train[t_f_xf] = X_train[t_f_xf].astype(int)

In [ ]:
## Verifying results

cf.report_df(X_train[t_f_xf])

## Converting Price to Float 

In [ ]:
## Converting each value into a float for processing

try:
    X_train['price'] = X_train['price'].map(lambda price: price[1:].replace(',','')).astype('float')
    X_train['price'][0]
except Exception:
    print('\nValues are already processed and saved. No changes necessary')
    print(f"\nSample value: {X_train['price'][0]}")

In [ ]:
X_train['price'].describe()

## Converting Host_Since to Datetime

---

> Since the 'host_since' feature is clearly a date, I will convert it to the "datetime" datatype. This allows me to create a separate feature for how many years of activity for each host.

---

In [ ]:
X_train.head(2)

In [ ]:
X_train.loc[:,'host_since'] = pd.to_datetime(X_train.loc[:,'host_since'])
X_train['host_since']

In [ ]:
X_train["host_since"].describe(datetime_is_numeric=True)

### Creating "Years_Hosting"

In [ ]:
range(len(X_train["host_since"]) -1)

In [ ]:
X_train['years_hosting'] = X_train["host_since"].map(lambda x: 2021- int(str(x).split("-")[0]))
X_train['years_hosting']

In [ ]:
X_train['years_hosting'].value_counts()

In [ ]:
X_train['years_hosting'].describe()

---

> I successfully created the new feature to represent how long each host is active (up to 2021). I will be curious to see the impact of the years of experience on the overall rating at the end of my modeling process.

---

## Bathrooms_Text to Num_Bathrooms

---

> In the raw data, the original "bathrooms" feature was empty and was dropped as part of processing missing data.
>
> **My goal is to convert the "bathrooms_text" feature into a new "num_bathrooms" feature to indicate the number of bathrooms at a host property.**
>
> I assume the number of bathrooms would have an impact on the rating . More bathrooms could mean more space/comfort for the guest, but could also cause an increase in price.


---

In [ ]:
## Checking current dataframe contents

X_train.head(3)

In [ ]:
## Checking for null values overall
X_train.isna().sum()[X_train.isna().sum() > 0]

In [ ]:
## Inspecting a selection of values from the column to understand the values
X_train.loc[:,'bathrooms_text'][:21]

In [ ]:
## Inspecting the rows in which there are null values
X_train[X_train['bathrooms_text'].isna()]

In [ ]:
## Filling null values with unique string ('Baths' not present otherwise)
## Unique string can be used later to check for any other zero baths

X_train.loc[:,'bathrooms_text'].fillna('0 Baths', inplace=True)

In [ ]:
## Verifying all null values are filled
X_train.isna().sum()[X_train.isna().sum() > 0]

In [ ]:
X_train.loc[:,'bathrooms_text'].isna().sum()

In [ ]:
## Splitting each list into separate strings
X_train['num_bathrooms'] = X_train['bathrooms_text'].map(lambda x: x.split(' ')[0])
X_train['num_bathrooms'].value_counts()

In [ ]:
## Inspecting results that are phrases, not numbers

replace = ['Half-bath', 'Shared', 'Private']

for x in X_train['bathrooms_text']:
    for i in replace:
        if i in x:
            print(x)

---

> **I will replace these values with the numeric value .5 as they are half-baths.** This will allow me to convert the column datatype to a float and use the column more easily in my modeling.

---

In [ ]:
## Replacing string values with .5 to represent half-bathrooms

replace = {'Half-bath': .5, 'Shared': .5, 'Private': .5}

X_train['num_bathrooms'].replace(replace, inplace = True)

X_train['num_bathrooms'] = X_train['num_bathrooms'].astype(float)

In [ ]:
## Inspecting resulting values

X_train['num_bathrooms'].value_counts(dropna=False)

In [ ]:
## Inspecting listings with more than 10 rooms

X_train[X_train['num_bathrooms'] >10]

---

> After taking a look at the locations listed above on Google Maps (using their latitude/longitude), I feel like these three listings with more than 10 bathrooms are either duplicates or incorrect values (for 50 baths).
>
> Due to the questionable nature of these values, I will drop these rows to prevent these outliers from impacting my results.

---

In [ ]:
## Inspecting rows where 'num_bathrooms' is zero to validate data

X_train[X_train['num_bathrooms'] ==0]

In [ ]:
## Removing old column post-conversion

X_train = X_train.drop(columns = 'bathrooms_text')

In [ ]:
## Confirming removal

'bathrooms_text' in X_train.columns

---

> My review of the original bathroom text for the zero bathrooms column shows that the listings are associated with a private room. This would make sense as the listings may not include an option such as a shared bath, etc..
>
> Additionally I did fill 9 instances of missing values with "0 Baths," which would contribute slightly to this count.
>
> Overall, I feel the data is valid and I will use it for my modeling.

---

## Binarizing Neighbourhood_Cleansed

---

> The current values for "neighbourhood_cleansed" are a single string value. **I will separate each neighborhood and convert them into a binary column to represent whether or not that neighborhood is included in the listing, then drop the old column.**

---

In [ ]:
X_train.loc[:,'neighbourhood_cleansed']

In [ ]:
X_train.loc[:,'neighbourhood_cleansed'].dtype

In [ ]:
## Testing the splitting between neighborhoods

X_train.loc[:,'neighbourhood_cleansed'][1].split(', ')

In [ ]:
## Converting values into a list of strings for each neighborhood

try:
    X_train['neighbourhood_cleansed'] = X_train['neighbourhood_cleansed'] \
                                                .apply(lambda x: x.split(', '))
    display(X_train.loc[:,'neighbourhood_cleansed'])
except Exception:
    print('\nValues are already processed and saved. No changes necessary.')
    print(f"\nSample value: {X_train.loc[:,'neighbourhood_cleansed'][3]}")

---

> The following code snippet is adapted from [here](https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list#:~:text=Sparse%20solution%20(for%20Pandas%20v0.25.0%2B)) by the user [Maxu](https://stackoverflow.com/users/5741205/maxu).

---

In [ ]:
## Converting each neighborhood into a binary column and dropping old column

mlb = MultiLabelBinarizer()

try:
    X_train = X_train.join(pd.DataFrame(mlb.fit_transform(X_train.pop('neighbourhood_cleansed')),
                              columns=mlb.classes_,index=X_train.index))
except Exception:
        print('\nValues are already processed and saved. No changes necessary.')

In [ ]:
## Inspecting results

X_train.head(3)

---

> After using the MultiLabelBinarizer, I successfully added a column for each neighborhood, indicating whether or not that neighborhood was included in the listing.
>
> This enables me to use the presence/absence of a  neighborhood as a category in my modeling.

---

## Host_Verifications to Binary Columns

In [ ]:
X_train['host_verifications'][:10]

---

> For the "host_verifications" and "amenities" features, the values are a single string with several items within the string.
>
> It is somewhat similar to the "neighborhoods_cleaned" feature in the sense that I will need to filter out the individual items from the string. However, there is an added complication as I need to remove the brackets and quotations from the strings.
>
> Once I filter out the items, I will be able to use the MultiLabelBinarizer again to create more categories for each amenity.

---

In [ ]:
## Testing the splitting between items

X_train.loc[:,'host_verifications'][1]

In [ ]:
for x in ['host_verifications', 'amenities']:
    X_train[x] = X_train[x].str.replace('[', '')
    X_train[x] = X_train[x].str.replace(']', '')
    X_train[x] = X_train[x].str.replace("'", '')
    X_train[x] = X_train[x].str.replace('"', '')
    X_train[x] = X_train[x].apply(lambda x: x.split(', '))

In [ ]:
## Converting each value into a binary column and dropping old column

mlb2 = MultiLabelBinarizer()
    
X_train = X_train.join(pd.DataFrame(mlb2.fit_transform(X_train.pop('host_verifications')),
                                  columns=mlb2.classes_,index=X_train.index))

X_train

---

> At this point, I successfully processed the 'host_verification' feature into distinct categories for modeling.
>
> In the future, I may attempt to do the same for the 'amenities' feature, but I don't want to create too many columns before my initial modeling.

---

## Converting Remaining Datatypes

In [ ]:
X_train.dtypes[:40]

## ❌ ERROR ❌ Binarizing Room_Type

---

> **Can't get MLB/OHE to work for individual property types.**

---

In [ ]:
# X_train['room_type'].describe()

In [ ]:
# X_train['room_type'].value_counts(dropna=False)

In [ ]:
# X_train['room_type'] = X_train['room_type'].replace('Entire home/apt', 'Home/Apt')

In [ ]:
# X_train['room_type'] = X_train['room_type'].map(lambda x: x.split(' ')[0])

In [ ]:
# X_train['room_type'].value_counts(dropna=False)

In [ ]:
# ohe = OneHotEncoder(sparse=False)

# X_train_ohe = ohe.fit_transform([X_train['room_type']])
# X_train_ohe

In [ ]:
# pd.DataFrame(X_train_ohe)

## ❌ ERROR ❌ Converting Amenities

---

> same issue as w/ room type

---

In [ ]:
# for x in ['host_verifications', 'amenities']:
#     print(X_train[x])

In [ ]:
# X_train['amenities'][:10]

In [ ]:
# for x in ['host_verifications', 'amenities']:
#     X_train[x] = X_train[x].str.replace('and', '')

In [ ]:
# ## Converting each value into a binary column and dropping old column

# mlb = MultiLabelBinarizer()
    
# X_train = X_train.join(pd.DataFrame(mlb.fit_transform(X_train.pop('amenities')),
#                                   columns=mlb.classes_,index=X_train.index))

In [ ]:
# X_train.loc[:,'host_verifications'] = X_train.loc[:,'host_verifications'].str.replace('[', '')
# X_train.loc[:,'host_verifications'] = X_train.loc[:,'host_verifications'].str.replace(']', '')
# X_train.loc[:,'host_verifications'] = X_train.loc[:,'host_verifications'].str.replace("'", '')

In [ ]:
# X_train.loc[:,'host_verifications']

In [ ]:
# X_train['amenities'] = X_train['amenities'].str.replace('[', '')
# X_train['amenities'] = X_train['amenities'].str.replace(']', '')
# X_train['amenities'] = X_train['amenities'].str.replace('"', '')

In [ ]:
# X_train['amenities']

In [ ]:
# X_train['amenities'] = X_train['amenities'].apply(lambda x: x.split(', '))

In [ ]:
# X_train['amenities'][0]

In [ ]:
# X_train['host_verifications'] = X_train['host_verifications'].apply(lambda x: x.split(', '))

In [ ]:
# X_train['host_verifications'][0][0]

In [ ]:
# def convert_to_col(X_train, list_cols):
#     '''For a given list of column names, separates each string value in the
#     column by the comma/space pattern to return new strings of single values.
    
#     Then, instantiates a MultiLabelBinarizer to create new columns for each 
#     new string to indicate the presence or absence of that string in the 
#     original column.'''
    
# #     mlb = MultiLabelBinarizer()
    
#     for x in list_cols:
#         try:
#             X_train[x] = X_train[x].apply(lambda x: x.split(', '))
#             print(f'Successfully split values in column "{x}"')
            
#         except Exception:
#             print('\nValues are already processed and saved.')
#             print(f"\nSample value: {X_train.loc[:,x][3]}")
            
# #         try:
# #             X_train = X_train.join(pd.DataFrame(mlb.fit_transform(X_train.pop(x)),
# #                                       columns=mlb.classes_,index=X_train.index))
# #         except Exception:
# #                 print('\nValues are already processed and saved.')
                
#     return X_train

In [ ]:
# binarize_cols = ['host_verifications', 'amenities'] 

# convert_to_col(X_train, binarize_cols)

In [ ]:
# ## Converting each value into a binary column and dropping old column

# mlb = MultiLabelBinarizer()
    
# X_train = X_train.join(pd.DataFrame(mlb.fit_transform(X_train.pop('amenities')),
#                                   columns=mlb.classes_,index=X_train.index))

In [ ]:
# # mlb = MultiLabelBinarizer()
    
# X_train = X_train.join(pd.DataFrame(mlb.fit_transform(X_train.pop('amenities')),
#                                   columns=mlb.classes_,index=X_train.index))

In [ ]:
# ## Converting values into a list of strings for each neighborhood

# try:
#     X_train['host_verifications'] = X_train['host_verifications'] \
#                                                 .apply(lambda x: x.split(', '))
#     display(X_train.loc[:,'host_verifications'])
# except Exception:
#     print('\nValues are already processed and saved. No changes necessary.')
#     print(f"\nSample value: {X_train.loc[:,'host_verifications'][3]}")
    
    

In [ ]:
# ## Inspecting results

# X_train.head(3)

In [ ]:
# test3 = X_train['host_verifications'][0]
# test3[1:-1].replace('"', "'").split(",")

In [ ]:
# # X_train['Tags'] = X_train.Tags.apply(lambda x: x[1:-1].split(','))

# X_train['host_verifications'].apply(lambda x: x.split(','))[0]

# Pre-Pipeline Review

In [ ]:
X_train.head(5)

In [ ]:
X_train.drop(columns = ['host_since', 'host_neighbourhood', 'amenities'], inplace=True)

In [ ]:
X_train.describe()

In [ ]:
X_train.isna().sum()[X_train.isna().sum() > 0]

In [ ]:
X_train[.isna().sum()

# **Transforming X_test**

## Converting True/False Columns to Binary Values

In [ ]:
## Creating list of true/false features to convert to 1/0, respectively

t_f_xf = ['host_is_superhost','host_has_profile_pic','host_identity_verified',
          'has_availability','instant_bookable']
t_f_xf

In [ ]:
## Converting datatype to "string" to replace values

X_train[t_f_xf] = X_train[t_f_xf].astype('str')
X_train[t_f_xf].dtypes

In [ ]:
X_train[t_f_xf]

In [ ]:
## Converting t/f to 1/0, respectively

X_train[t_f_xf] = X_train[t_f_xf].replace({ 't' : 1, 'f' : 0})

In [ ]:
X_train[t_f_xf]

In [ ]:
X_train[t_f_xf] = X_train[t_f_xf].astype(int)

In [ ]:
## Verifying results

cf.report_df(X_train[t_f_xf])

## Converting Price to Float 

In [ ]:
## Converting each value into a float for processing

try:
    X_train['price'] = X_train['price'].map(lambda price: price[1:].replace(',','')).astype('float')
    X_train['price'][0]
except Exception:
    print('\nValues are already processed and saved. No changes necessary')
    print(f"\nSample value: {X_train['price'][0]}")

In [ ]:
X_train['price'].describe()

## Converting Host_Since to Datetime

---

> Since the 'host_since' feature is clearly a date, I will convert it to the "datetime" datatype. This allows me to create a separate feature for how many years of activity for each host.

---

In [ ]:
X_train.head(2)

In [ ]:
X_train.loc[:,'host_since'] = pd.to_datetime(X_train.loc[:,'host_since'])
X_train['host_since']

In [ ]:
X_train["host_since"].describe(datetime_is_numeric=True)

### Creating "Years_Hosting"

In [ ]:
range(len(X_train["host_since"]) -1)

In [ ]:
X_train['years_hosting'] = X_train["host_since"].map(lambda x: 2021- int(str(x).split("-")[0]))
X_train['years_hosting']

In [ ]:
X_train['years_hosting'].value_counts()

In [ ]:
X_train['years_hosting'].describe()

---

> I successfully created the new feature to represent how long each host is active (up to 2021). I will be curious to see the impact of the years of experience on the overall rating at the end of my modeling process.

---

## Bathrooms_Text to Num_Bathrooms

---

> In the raw data, the original "bathrooms" feature was empty and was dropped as part of processing missing data.
>
> **My goal is to convert the "bathrooms_text" feature into a new "num_bathrooms" feature to indicate the number of bathrooms at a host property.**
>
> I assume the number of bathrooms would have an impact on the rating . More bathrooms could mean more space/comfort for the guest, but could also cause an increase in price.


---

In [ ]:
## Checking current dataframe contents

X_train.head(3)

In [ ]:
## Checking for null values overall
X_train.isna().sum()[X_train.isna().sum() > 0]

In [ ]:
## Inspecting a selection of values from the column to understand the values
X_train.loc[:,'bathrooms_text'][:21]

In [ ]:
## Inspecting the rows in which there are null values
X_train[X_train['bathrooms_text'].isna()]

In [ ]:
## Filling null values with unique string ('Baths' not present otherwise)
## Unique string can be used later to check for any other zero baths

X_train.loc[:,'bathrooms_text'].fillna('0 Baths', inplace=True)

In [ ]:
## Verifying all null values are filled
X_train.isna().sum()[X_train.isna().sum() > 0]

In [ ]:
X_train.loc[:,'bathrooms_text'].isna().sum()

In [ ]:
## Splitting each list into separate strings
X_train['num_bathrooms'] = X_train['bathrooms_text'].map(lambda x: x.split(' ')[0])
X_train['num_bathrooms'].value_counts()

In [ ]:
## Inspecting results that are phrases, not numbers

replace = ['Half-bath', 'Shared', 'Private']

for x in X_train['bathrooms_text']:
    for i in replace:
        if i in x:
            print(x)

---

> **I will replace these values with the numeric value .5 as they are half-baths.** This will allow me to convert the column datatype to a float and use the column more easily in my modeling.

---

In [ ]:
## Replacing string values with .5 to represent half-bathrooms

replace = {'Half-bath': .5, 'Shared': .5, 'Private': .5}

X_train['num_bathrooms'].replace(replace, inplace = True)

X_train['num_bathrooms'] = X_train['num_bathrooms'].astype(float)

In [ ]:
## Inspecting resulting values

X_train['num_bathrooms'].value_counts(dropna=False)

In [ ]:
## Inspecting listings with more than 10 rooms

X_train[X_train['num_bathrooms'] >10]

---

> After taking a look at the locations listed above on Google Maps (using their latitude/longitude), I feel like these three listings with more than 10 bathrooms are either duplicates or incorrect values (for 50 baths).
>
> Due to the questionable nature of these values, I will drop these rows to prevent these outliers from impacting my results.

---

In [ ]:
## Inspecting rows where 'num_bathrooms' is zero to validate data

X_train[X_train['num_bathrooms'] ==0]

In [ ]:
## Removing old column post-conversion

X_train = X_train.drop(columns = 'bathrooms_text')

In [ ]:
## Confirming removal

'bathrooms_text' in X_train.columns

---

> My review of the original bathroom text for the zero bathrooms column shows that the listings are associated with a private room. This would make sense as the listings may not include an option such as a shared bath, etc..
>
> Additionally I did fill 9 instances of missing values with "0 Baths," which would contribute slightly to this count.
>
> Overall, I feel the data is valid and I will use it for my modeling.

---

## Binarizing Neighbourhood_Cleansed

---

> The current values for "neighbourhood_cleansed" are a single string value. **I will separate each neighborhood and convert them into a binary column to represent whether or not that neighborhood is included in the listing, then drop the old column.**

---

In [ ]:
X_train.loc[:,'neighbourhood_cleansed']

In [ ]:
X_train.loc[:,'neighbourhood_cleansed'].dtype

In [ ]:
## Testing the splitting between neighborhoods

X_train.loc[:,'neighbourhood_cleansed'][1].split(', ')

In [ ]:
## Converting values into a list of strings for each neighborhood

try:
    X_train['neighbourhood_cleansed'] = X_train['neighbourhood_cleansed'] \
                                                .apply(lambda x: x.split(', '))
    display(X_train.loc[:,'neighbourhood_cleansed'])
except Exception:
    print('\nValues are already processed and saved. No changes necessary.')
    print(f"\nSample value: {X_train.loc[:,'neighbourhood_cleansed'][3]}")

---

> The following code snippet is adapted from [here](https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list#:~:text=Sparse%20solution%20(for%20Pandas%20v0.25.0%2B)) by the user [Maxu](https://stackoverflow.com/users/5741205/maxu).

---

In [ ]:
## Converting each neighborhood into a binary column and dropping old column

mlb = MultiLabelBinarizer()

try:
    X_train = X_train.join(pd.DataFrame(mlb.fit_transform(X_train.pop('neighbourhood_cleansed')),
                              columns=mlb.classes_,index=X_train.index))
except Exception:
        print('\nValues are already processed and saved. No changes necessary.')

In [ ]:
## Inspecting results

X_train.head(3)

---

> After using the MultiLabelBinarizer, I successfully added a column for each neighborhood, indicating whether or not that neighborhood was included in the listing.
>
> This enables me to use the presence/absence of a  neighborhood as a category in my modeling.

---

## Host_Verifications to Binary Columns

In [ ]:
X_train['host_verifications'][:10]

---

> For the "host_verifications" and "amenities" features, the values are a single string with several items within the string.
>
> It is somewhat similar to the "neighborhoods_cleaned" feature in the sense that I will need to filter out the individual items from the string. However, there is an added complication as I need to remove the brackets and quotations from the strings.
>
> Once I filter out the items, I will be able to use the MultiLabelBinarizer again to create more categories for each amenity.

---

In [ ]:
## Testing the splitting between items

X_train.loc[:,'host_verifications'][1]

In [ ]:
for x in ['host_verifications', 'amenities']:
    X_train[x] = X_train[x].str.replace('[', '')
    X_train[x] = X_train[x].str.replace(']', '')
    X_train[x] = X_train[x].str.replace("'", '')
    X_train[x] = X_train[x].str.replace('"', '')
    X_train[x] = X_train[x].apply(lambda x: x.split(', '))

In [ ]:
## Converting each value into a binary column and dropping old column

mlb2 = MultiLabelBinarizer()
    
X_train = X_train.join(pd.DataFrame(mlb2.fit_transform(X_train.pop('host_verifications')),
                                  columns=mlb2.classes_,index=X_train.index))

X_train

---

> At this point, I successfully processed the 'host_verification' feature into distinct categories for modeling.
>
> In the future, I may attempt to do the same for the 'amenities' feature, but I don't want to create too many columns before my initial modeling.

---

## Converting Remaining Datatypes

In [ ]:
X_train.dtypes[:40]

# 🚿 **Preprocessing Pipeline**

In [ ]:
num_cols = X_train.select_dtypes(include=[int, float]).columns.to_list()
# num_cols

In [ ]:
cat_cols = ['room_type']
cat_cols

In [ ]:
X_train[cat_cols] = X_train[cat_cols].astype(str)

In [ ]:
# X_test[cat_cols] = X_test[cat_cols].astype(str)

In [ ]:
## Creating ColumnTransformer and sub-transformers for imputation and encoding

# Filling missing values in "Beds" and "Bedrooms"
miss_num_transformer = SimpleImputer(strategy='mean')

# Filling missing values in "Beds" and "Bedrooms"
miss_cat_transformer = SimpleImputer(strategy='most_frequent')

## Encoding categoricals - handling errors to prevent issues w/ test set
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

cat_pipe = Pipeline(steps=[('imputer', miss_cat_transformer),
                      ('ohe', categorical_transformer)])

num_pipe = Pipeline(steps=[('imputer', miss_num_transformer),
                           ('scaler', StandardScaler())])

## Instantiating the ColumnTransformer and including all transformers
preprocessor = ColumnTransformer(
    transformers=[('nums', num_pipe, num_cols),
                  ('cats', cat_pipe, cat_cols)])

preprocessor

In [ ]:
preprocessor.fit(X_train)

## Getting feature names from OHE
ohe_cat_names = preprocessor.named_transformers_['cats'].named_steps['ohe'].get_feature_names(cat_cols)

In [ ]:
## Generating list for column index
final_cols = [*num_cols, *ohe_cat_names]

## Fit and transform the data via the ColumnTransformer
X_train_tf = preprocessor.transform(X_train)
X_train_tf_df = pd.DataFrame(X_train_tf, columns=final_cols, index=X_train.index)

## Transforming the test set and saving
X_test_tf = preprocessor.transform(X_test)
X_test_tf_df = pd.DataFrame(X_test_tf, columns=final_cols, index=X_test.index)

display(X_train_tf_df.head(5),X_test_tf_df.head(5))

# 📝 Next Steps

* Process classification model - i.e. Logreg, KNN, DecisionTrees, etc.
* Evaluate results
* Determine if I need to redo pre-processing steps

# 🚿 Classification Pipeline